# Support Vector Machines

### References

- https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
- https://scikit-learn.org/stable/modules/svm.html#classification

## Summary

1. We use RBF kernel for SVM model
2. Run Grid Search CV the best parameters are: 
- `C` = 1000
- `gamma` = 0.0001
3. With preprocessing
- `evaluation metric = 0.212`
4. With feature engineering, we create `55` new features with `FeatureTools`
- `evaluation metric = 0.179`

In [112]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [113]:
training = pd.read_csv('train.csv')
testing = pd.read_csv('test.csv')

### 0. Data Preprocessing

In [114]:
#dropping columns
df1 = training.copy()
df1.drop(['PetID', 'Description', 'RescuerID'], axis=1, inplace=True)

df2 = testing.copy()
df2.drop(['PetID', 'Description','RescuerID'], axis=1, inplace=True)


#change Name to 1 - has name, 0 - has no name
df1['Name'] = df1['Name'].fillna(0)
df1['Name'].replace({"No Name": 0, "No Name Yet": 0, "Unknown": 0},inplace=True)
df1.loc[df1['Name'] !=0, 'Name'] = 1

df2['Name'] = df2['Name'].fillna(0)
df2['Name'].replace({"No Name": 0, "No Name Yet": 0, "Unknown": 0},inplace=True)
df2.loc[df2['Name'] !=0, 'Name'] = 1

In [115]:
#outlier removal
    #print(df1['Age'].describe())
df1 = df1[(np.abs(stats.zscore(df1['Age'])) < 3)]
    #print(df1['Age'].describe())
df1 = df1[(np.abs(stats.zscore(df1['PhotoAmt'])) < 3)]
df1 = df1[(np.abs(stats.zscore(df1['Quantity'])) < 3)]

df2 = df2[(np.abs(stats.zscore(df2['Age'])) < 3)]
    #print(df1['Age'].describe())
df2 = df2[(np.abs(stats.zscore(df2['PhotoAmt'])) < 3)]
df2 = df2[(np.abs(stats.zscore(df2['Quantity'])) < 3)]

In [116]:
#normalise numerical columns
columns_to_normalize = ['Age','MaturitySize','FurLength','Quantity','PhotoAmt','Fee', 'VideoAmt']
x1 = df1[columns_to_normalize].values
scaler = MinMaxScaler()

x1_scaled = scaler.fit_transform(x1)
df_temp1 = pd.DataFrame(x1_scaled, columns=columns_to_normalize, index = df1.index)
df1[columns_to_normalize] = df_temp1

x2 = df2[columns_to_normalize].values
x2_scaled = scaler.fit_transform(x2)
df_temp2 = pd.DataFrame(x2_scaled, columns=columns_to_normalize, index = df2.index)
df2[columns_to_normalize] = df_temp2

In [118]:
df2

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
0,2,1,0.133333,266,266,1,2,6,7,0.000000,0.0,1,1,2,1,0.2,0.000000,41326,0.0,0.153846
1,2,1,0.600000,285,264,2,1,4,7,0.333333,1.0,1,1,1,2,0.0,0.000000,41326,0.0,0.076923
2,2,1,0.033333,265,0,1,6,7,0,0.333333,0.5,1,1,2,1,0.0,0.133333,41326,0.0,0.307692
3,1,1,0.200000,307,0,2,2,5,0,0.333333,0.5,1,1,1,1,0.2,0.000000,41326,0.0,0.384615
4,2,1,0.200000,265,0,1,2,3,7,0.333333,0.5,1,1,1,1,0.0,0.000000,41326,0.0,0.384615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3967,1,1,0.083333,307,307,1,1,2,0,0.333333,0.0,1,1,2,1,0.2,0.100000,41401,0.0,0.384615
3968,1,1,0.500000,307,307,1,1,2,7,0.333333,0.0,3,1,2,1,0.0,0.000000,41326,0.0,0.153846
3969,2,0,0.083333,300,0,3,1,2,4,0.333333,0.5,2,3,2,1,1.0,0.000000,41401,0.0,0.076923
3970,1,1,0.400000,307,0,2,5,7,0,0.333333,0.5,2,2,2,1,0.0,0.000000,41325,0.0,0.230769


In [119]:
df1

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
0,2,1,0.046875,299,0,1,1,7,0,0.000000,...,2,2,2,1,0.00,0.033333,41326,0.0,0.071429,2
1,2,0,0.015625,265,0,1,1,2,0,0.333333,...,3,3,3,1,0.00,0.000000,41401,0.0,0.142857,0
2,1,1,0.015625,307,0,1,2,7,0,0.333333,...,1,1,2,1,0.00,0.000000,41326,0.0,0.500000,3
3,1,1,0.062500,307,0,2,1,2,0,0.333333,...,1,1,2,1,0.00,0.050000,41401,0.0,0.571429,2
4,1,1,0.015625,307,0,1,1,0,0,0.333333,...,2,2,2,1,0.00,0.000000,41326,0.0,0.214286,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,0,0.031250,266,0,3,1,0,0,0.333333,...,2,2,2,1,0.75,0.000000,41326,0.0,0.214286,2
14989,2,1,0.937500,265,264,3,1,4,7,0.333333,...,1,1,1,1,0.25,0.000000,41326,0.0,0.214286,4
14990,2,1,0.031250,265,266,3,5,6,7,0.666667,...,2,1,3,1,1.00,0.010000,41326,0.0,0.357143,3
14991,2,1,0.140625,266,0,2,4,7,0,0.000000,...,1,1,1,1,0.00,0.000000,41336,0.0,0.214286,4


### Splitting Data

In [120]:
X_features = df1.iloc[:, :-1]
y_val = df1.iloc[:, -1:]

X_train, X_test, y_train, y_test = train_test_split(X_features, y_val, test_size = 0.2)

## 1. SVM Model

 Hyperparameters:
1. `C`: 
- C is the penalty parameter, which represents misclassification or error term, used to maintain regularization. 
- The misclassification or error term tells the SVM optimization how much error is bearable. This is how you can control the trade-off between decision boundary and misclassification term. 
- A smaller value of C creates a small-margin hyperplane and a larger value of C creates a larger-margin hyperplane.
2. `kernel`: 
- Kernel transforms the given dataset input data into the required form - linear, polynomial, and radial basis function (RBF). 
- Polynomial and RBF are useful for non-linear hyperplane. Polynomial and RBF kernels compute the separation line in the higher dimension. 
3. Gamma `γ` : 
- A lower value of Gamma will loosely fit the training dataset, whereas a higher value of gamma will exactly fit the training dataset, which causes over-fitting.

### Linear Kernel

In [40]:
clf_linear = svm.SVC(kernel='linear')

In [12]:
clf_linear.fit(X_train, y_train.values.ravel())
pred_linear = clf_linear.predict(X_test)

In [16]:
print('Accuracy: ', metrics.accuracy_score(y_test.values, pred_linear))

Accuracy:  0.2205091430620294


In [32]:
print('Evaluation metric: ', cohen_kappa_score(y_test.to_numpy().ravel(), pred_linear, weights='quadratic'))

Evaluation metric:  -0.014228816908305575


### Radial basis function Kernel

This kernel is the most successful kernel, due to the flexibility of separating observations with this method. Additionally to the cost parameter `C`, the hyperparameter `γ` has to be tuned.

#### Hyperparameter Tuning

In [43]:
# defining parameter range
quadratic_kappa_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3, scoring=quadratic_kappa_scorer)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  12.1s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 2/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=  11.9s


...

[CV 4/5] END ...............C=1000, gamma=0.0001, kernel=rbf; total time=  31.4s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[CV 5/5] END ...............C=1000, gamma=0.0001, kernel=rbf; total time=  29.5s


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             scoring=make_scorer(cohen_kappa_score, weights=quadratic),
             verbose=3)

In [44]:
print(grid.best_params_)

{'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}


In [57]:
clf_rbf = svm.SVC(C=1000, gamma=0.0001, kernel='rbf')

In [58]:
clf_rbf.fit(X_train, y_train.values.ravel())
pred_rbf = clf_rbf.predict(X_test)

#### Evaluation

In [59]:
print('Accuracy: ', metrics.accuracy_score(y_test, pred_rbf))

Accuracy:  0.33416995338831124


In [60]:
print('Evaluation metric: ', cohen_kappa_score(y_test.to_numpy().ravel(), pred_rbf, weights='quadratic'))

Evaluation metric:  0.21214329683399757


## 2. Feature Engineering

- Through `featuretools` Python package

In [121]:
import featuretools as ft
import featuretools.variable_types as vtypes

In [128]:
df_fe = df1.copy()
df_fe.drop(['AdoptionSpeed'], axis=1, inplace=True)

In [129]:
df_fe

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt
0,2,1,0.046875,299,0,1,1,7,0,0.000000,0.0,2,2,2,1,0.00,0.033333,41326,0.0,0.071429
1,2,0,0.015625,265,0,1,1,2,0,0.333333,0.5,3,3,3,1,0.00,0.000000,41401,0.0,0.142857
2,1,1,0.015625,307,0,1,2,7,0,0.333333,0.5,1,1,2,1,0.00,0.000000,41326,0.0,0.500000
3,1,1,0.062500,307,0,2,1,2,0,0.333333,0.0,1,1,2,1,0.00,0.050000,41401,0.0,0.571429
4,1,1,0.015625,307,0,1,1,0,0,0.333333,0.0,2,2,2,1,0.00,0.000000,41326,0.0,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14988,2,0,0.031250,266,0,3,1,0,0,0.333333,0.5,2,2,2,1,0.75,0.000000,41326,0.0,0.214286
14989,2,1,0.937500,265,264,3,1,4,7,0.333333,0.5,1,1,1,1,0.25,0.000000,41326,0.0,0.214286
14990,2,1,0.031250,265,266,3,5,6,7,0.666667,0.5,2,1,3,1,1.00,0.010000,41326,0.0,0.357143
14991,2,1,0.140625,266,0,2,4,7,0,0.000000,0.0,1,1,1,1,0.00,0.000000,41336,0.0,0.214286


#### Set up entitysets

In [130]:
es = ft.EntitySet()

var_types = {'Type':vtypes.Categorical, 
             'Breed1': vtypes.Categorical, 
             'Breed2': vtypes.Categorical, 
             'Gender':vtypes.Categorical, 
             'Color1': vtypes.Categorical, 
             'Color2': vtypes.Categorical, 
             'Color3': vtypes.Categorical, 
             'MaturitySize': vtypes.Categorical, 
             'FurLength': vtypes.Categorical, 
             'Vaccinated': vtypes.Categorical, 
             'Dewormed': vtypes.Categorical, 
             'Sterilized': vtypes.Categorical, 
             'Health': vtypes.Categorical, 
             'State': vtypes.Categorical}

es.entity_from_dataframe(entity_id='pet', dataframe=df_fe, make_index=True, index = 'pet_index', variable_types=var_types)

Entityset: None
  Entities:
    pet [Rows: 13944, Columns: 21]
  Relationships:
    No relationships

In [133]:
es.normalize_entity(base_entity_id='pet', new_entity_id='PhotoAmt', index='PhotoAmt')

Entityset: None
  Entities:
    pet [Rows: 13944, Columns: 21]
    PhotoAmt [Rows: 15, Columns: 1]
  Relationships:
    pet.PhotoAmt -> PhotoAmt.PhotoAmt

In [131]:
#verify that X features are of correct data type
es['pet'].variables

[<Variable: pet_index (dtype = index)>,
 <Variable: Name (dtype = categorical)>,
 <Variable: Age (dtype = numeric)>,
 <Variable: Quantity (dtype = numeric)>,
 <Variable: Fee (dtype = numeric)>,
 <Variable: VideoAmt (dtype = numeric)>,
 <Variable: PhotoAmt (dtype = numeric)>,
 <Variable: Type (dtype = categorical)>,
 <Variable: Breed1 (dtype = categorical)>,
 <Variable: Breed2 (dtype = categorical)>,
 <Variable: Gender (dtype = categorical)>,
 <Variable: Color1 (dtype = categorical)>,
 <Variable: Color2 (dtype = categorical)>,
 <Variable: Color3 (dtype = categorical)>,
 <Variable: MaturitySize (dtype = categorical)>,
 <Variable: FurLength (dtype = categorical)>,
 <Variable: Vaccinated (dtype = categorical)>,
 <Variable: Dewormed (dtype = categorical)>,
 <Variable: Sterilized (dtype = categorical)>,
 <Variable: Health (dtype = categorical)>,
 <Variable: State (dtype = categorical)>]

#### Run Deep Feature Synthesis (creating new features)

- `55` new features created

In [134]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
    target_entity = 'pet', 
    max_depth = 3, 
    verbose = 3, 
    n_jobs = 1)

Built 75 features
Elapsed: 00:00 | Progress: 100%|██████████


In [148]:
feature_matrix

,Name,Age,Quantity,Fee,VideoAmt,PhotoAmt,Type,Breed1,Breed2,Gender,...,PhotoAmt.SKEW(pet.Quantity),PhotoAmt.SKEW(pet.VideoAmt),PhotoAmt.STD(pet.Age),PhotoAmt.STD(pet.Fee),PhotoAmt.STD(pet.Quantity),PhotoAmt.STD(pet.VideoAmt),PhotoAmt.SUM(pet.Age),PhotoAmt.SUM(pet.Fee),PhotoAmt.SUM(pet.Quantity),PhotoAmt.SUM(pet.VideoAmt)
pet_index,,,,,,,,,,,,,,,,,,,,,
0,1,0.046875,0.00,0.033333,0.0,0.071429,2,299,0,1,...,3.320262,9.506369,0.194989,0.022330,0.197761,0.018966,403.625000,18.064667,192.00,6.000000
1,0,0.015625,0.00,0.000000,0.0,0.142857,2,265,0,1,...,3.210893,8.328495,0.208845,0.025245,0.170300,0.027206,376.500000,17.449000,153.25,9.166667
2,1,0.015625,0.00,0.000000,0.0,0.500000,1,307,0,1,...,2.052613,7.424463,0.144137,0.023975,0.266561,0.082502,38.921875,2.279000,54.25,7.833333
3,1,0.062500,0.00,0.050000,0.0,0.571429,1,307,0,2,...,1.749386,3.648905,0.162102,0.022755,0.287688,0.074611,29.765625,2.397667,46.50,7.000000
4,1,0.015625,0.00,0.000000,0.0,0.214286,1,307,0,1,...,2.590054,9.669748,0.191303,0.028536,0.203326,0.032838,331.671875,18.869667,209.00,9.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13939,0,0.031250,0.75,0.000000,0.0,0.214286,2,266,0,3,...,2.590054,9.669748,0.191303,0.028536,0.203326,0.032838,331.671875,18.869667,209.00,9.833333
13940,1,0.937500,0.25,0.000000,0.0,0.214286,2,265,264,3,...,2.590054,9.669748,0.191303,0.028536,0.203326,0.032838,331.671875,18.869667,209.00,9.833333
13941,1,0.031250,1.00,0.010000,0.0,0.357143,2,265,266,3,...,1.954923,7.980785,0.179706,0.034448,0.271512,0.047605,251.234375,16.314667,285.00,17.166667


In [143]:
fe = feature_matrix.copy()
print(fe.shape)
print(y_val.shape)

(13944, 75)
(13944, 1)


#### Run SVM with this new X features

In [139]:
X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(fe, y_val, test_size = 0.2)

In [147]:
y_val

,AdoptionSpeed
0,2
1,0
2,3
3,2
4,2
...,...
14988,2
14989,4
14990,3
14991,4


In [141]:
X_train_fe

,Name,Age,Quantity,Fee,VideoAmt,PhotoAmt,Type,Breed1,Breed2,Gender,...,PhotoAmt.SKEW(pet.Quantity),PhotoAmt.SKEW(pet.VideoAmt),PhotoAmt.STD(pet.Age),PhotoAmt.STD(pet.Fee),PhotoAmt.STD(pet.Quantity),PhotoAmt.STD(pet.VideoAmt),PhotoAmt.SUM(pet.Age),PhotoAmt.SUM(pet.Fee),PhotoAmt.SUM(pet.Quantity),PhotoAmt.SUM(pet.VideoAmt)
pet_index,,,,,,,,,,,,,,,,,,,,,
3953,1,0.078125,0.00,0.000000,0.0,0.357143,2,299,0,2,...,1.954923,7.980785,0.179706,0.034448,0.271512,0.047605,251.234375,16.314667,285.00,17.166667
9911,1,0.031250,0.00,0.000000,0.0,0.142857,2,266,0,2,...,3.210893,8.328495,0.208845,0.025245,0.170300,0.027206,376.500000,17.449000,153.25,9.166667
11614,0,0.031250,0.00,0.000000,0.0,0.071429,1,307,0,2,...,3.320262,9.506369,0.194989,0.022330,0.197761,0.018966,403.625000,18.064667,192.00,6.000000
12396,1,0.031250,0.50,0.000000,0.0,0.642857,2,303,0,3,...,1.928809,3.607385,0.164564,0.032112,0.250071,0.066666,21.562500,2.044000,29.00,4.166667
1923,0,0.187500,0.00,0.000000,0.0,0.071429,2,266,0,2,...,3.320262,9.506369,0.194989,0.022330,0.197761,0.018966,403.625000,18.064667,192.00,6.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,1,0.125000,0.00,0.000000,0.0,0.142857,1,307,0,1,...,3.210893,8.328495,0.208845,0.025245,0.170300,0.027206,376.500000,17.449000,153.25,9.166667
10049,1,0.031250,0.00,0.000000,0.0,0.000000,1,307,141,2,...,1.834784,17.635192,0.236906,0.025605,0.284815,0.009451,58.734375,2.233667,47.00,0.166667
7264,1,0.187500,0.25,0.000000,0.0,0.428571,1,307,0,1,...,2.293315,5.184472,0.166957,0.023248,0.250567,0.085553,64.828125,3.831667,69.00,12.833333


In [146]:
y_train_fe

,AdoptionSpeed
4260,1
10670,2
12492,4
13335,3
2081,1
...,...
4120,4
10818,0
7811,3
3338,3


In [149]:
clf_rbf.fit(X_train_fe, y_train_fe.values.ravel())
pred_rbf_fe = clf_rbf.predict(X_test_fe)

In [150]:
print('Evaluation metric: ', cohen_kappa_score(y_test_fe.to_numpy().ravel(), pred_rbf_fe, weights='quadratic'))

Evaluation metric:  0.1786678757468736


In [151]:
pred_rbf_fe

array([2, 2, 1, ..., 4, 3, 4])